<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/PR%C3%80CTICA_GUIADA/Crank_em_vull_matar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bueno toca Crank-Nicolson  \
Si fas com derivades numèriques i reordenes queda:
$T_{i+1}^n (1+2\gamma)-\gamma·T_{i+1}^{n+1}-\gamma · T_{i+1}^{n-1} = T_i^n + \gamma · T_i^{n+1} + \gamma ·T_i
^{n-1} + Δt$\
que es pot escriure com un sistema de matrius to jodido pero bueno tenim llibreries\
Tenim dues matrius tridiagonals.\
 La de l'esquerre, matriu A, te per offdiagonal $-\gamma$ i per diagonal $(1+2\gamma)$ i multiplica un vector de les T a temps i+1\
 La de la dreta, matriu B, te per offdiagonal $\gamma$ i per diagonal $(1-2\gamma)$ i multiplica un vector de les T a temps i i li suma $Δt$\
 Necessitem puto invertir la matriu de l'esquerre, multiplicar tot a la dreta per l'inversa i tenim el vector d'incognites aïllat amb coses que coneixem (vamos que tenim el sistema resolt)

In [43]:
import numpy as np
from scipy.linalg import toeplitz
from numpy.linalg import inv

#definim matriu A
C_v= 3686
rho= 1081
kappa= 0.56
sigma= 0.472
volts= 40

#vamo a calcular:
alpha_var= kappa/(C_v*rho) #es la difusivitat

N= 101   # Mallat espaial, nombre de punts que tenim
x_0= 0.02 #per la cara, pq x_f ha de ser 0.02 (2 cm) i ha d'estar normalitzat així que ha de valer 1 JA SABEM PER QUÈ ENS HEM D'INVENTAR x_0
x_f= 0.02/x_0 #normalitzar, ja que \tilde{x}=x/x_0
x_i= 0
t_i= 0
t_0= x_0**2/alpha_var #\tilde{t}=t/t_0
t_f= 0.025 #ja està normalitzat, ho diu l'enunciat

P_ext= volts**2*sigma/(2*(x_0)**2) #calculada per anàlisi dimensional a partir de dades proporcionades, el factor 2 dividint és perquè s'ha d'agafar el voltatge efectiu
T_0= P_ext/(C_v*rho)*t_0 #KELVIN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
print("T_0=",T_0,"k")
T_init= 309.65/T_0   #CREC QUE HA D'ESTAR EN KELVIN PER LES UNITATS YOQUESE
print("T_init (JA NORMALITZADA)=",T_init,"k")

dx=x_f/(N-1)
dt_1= dx**2
dt_2= 1/2*dx**2
def CN(N, gamma, T_i, t_f, dt, dx):

  dt_vect=np.ones((N-2,1))*dt
  dt_vect[0]= 2*gamma*T_i + dt
  dt_vect[-1]=2*gamma*T_i + dt

  T_i_vect = np.ones((N-2,1))*T_i
  fila1_A= (N-2)*[0]
  fila1_A [0]= 1+2*gamma
  fila1_A [1]= -gamma
  col_A = (N-2)*[0]
  col_A [1]= -gamma
  A = toeplitz(fila1_A,col_A)

  print(A)

  fila1_B= (N-2)*[0]
  fila1_B [0]= 1-2*gamma
  fila1_B [1]= gamma
  col_B = (N-2)*[0]
  col_B [1]= gamma
  B = toeplitz(fila1_B,col_B)

  print(B)
  Ainv=inv(A)
  t_i=0
  while t_i<t_f:
    T_i_vect= np.matmul(Ainv,np.matmul(B,T_i_vect))+np.matmul(Ainv,dt_vect)
    t_i = t_i +dt

  T_i_vect = T_i_vect*T_0-273.15
  T_i_vect =np.append(T_i_vect,[36.5])
  T_i_vect =np.append([36.5],T_i_vect)
  return print(T_i_vect,len(T_i_vect))

CN(101,1/2,T_init,t_f,dt_1,dx)
CN(101,1/4,T_init,t_f,dt_2,dx)



T_0= 674.2857142857143 k
T_init (JA NORMALITZADA)= 0.4592266949152542 k
[[ 2.  -0.5  0.  ...  0.   0.   0. ]
 [-0.5  2.  -0.5 ...  0.   0.   0. ]
 [ 0.  -0.5  2.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  2.  -0.5  0. ]
 [ 0.   0.   0.  ... -0.5  2.  -0.5]
 [ 0.   0.   0.  ...  0.  -0.5  2. ]]
[[0.  0.5 0.  ... 0.  0.  0. ]
 [0.5 0.  0.5 ... 0.  0.  0. ]
 [0.  0.5 0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.5 0. ]
 [0.  0.  0.  ... 0.5 0.  0.5]
 [0.  0.  0.  ... 0.  0.5 0. ]]
[36.5        37.67199428 38.77896037 39.82329387 40.80738081 41.73359301
 42.60428336 43.42178142 44.18838897 44.90637582 45.57797578 46.20538283
 46.79074751 47.33617355 47.84371477 48.3153722  48.75309152 49.15876075
 49.53420826 49.88120102 50.20144314 50.49657476 50.76817114 51.01774203
 51.24673135 51.45651704 51.64841124 51.8236606  51.98344683 52.12888744
 52.26103665 52.38088642 52.48936762 52.58735133 52.6756502  52.75501985
 52.8261604  52.88971791 52.94628594 52.996407   53.04057404 53.079231

In [ ]:
)